In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

df = pd.read_csv('churn_data.csv')
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


# Data Preparation:

* prepare data
* fill the missing value

In [2]:
# prepare columns
df.columns = df.columns.str.lower().str.replace(' ', '_')

#prepare data
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for column in categorical_columns:
    df[column] = df[column].str.lower().str.replace(' ', '_')

In [3]:
# convert column 'totalcharges' into numeric
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

In [4]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                object
dtype: object

In [5]:
df.churn = (df.churn == 'yes').astype(int)

# Setting up the validation framework

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
df_full_train , df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train , df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_full_train.shape , df_test.shape

((5634, 21), (1409, 21))

In [8]:
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)

In [9]:
y_train = df_train.churn.values
y_test = df_test.churn.values
y_val = df_val.churn.values

In [10]:
del df_train['churn']
del df_test['churn']
del df_val['churn']

# EDA 

* Check missing values
* Look at the target variable (churn)
* Look at numerical and categorical variables

In [11]:
df_full_train = df_full_train.reset_index(drop=True)

In [12]:
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [13]:
df_full_train.churn.value_counts(normalize = True)

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [14]:
df_full_train.churn.mean()

0.26996805111821087

In [15]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [16]:
numeric = ['tenure','monthlycharges','totalcharges' ]


categorical = ['gender', 'seniorcitizen', 'partner', 'dependents', 
        'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

# Feature Importance: churn rate and risk ratio

Feature importance analysis (part of EDA) - identifying which features affect our target variable

* Churn rate
* Risk ratio
* Mutual information - later

In [17]:
from IPython.display import display

In [18]:
global_churn_rate = df_full_train.churn.mean()

In [19]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

0.27682403433476394

In [20]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean','count'])
    df_group['diff'] = df_group['mean']- global_churn_rate 
    df_group['risk'] = df_group['mean']/ global_churn_rate
    display(df_group)
    print()
    

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980



seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208



partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730



paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560



paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


#  Feature importance: Mutual information

Mutual information - concept from information theory, it tells us how much we can learn about one variable if we know the value of another

* https://en.wikipedia.org/wiki/Mutual_information

In [21]:
from sklearn.metrics import mutual_info_score

In [22]:
def mutual_info_churn_score(Series):
    return mutual_info_score(df_full_train.churn, Series)

In [23]:
mutual_info = df_full_train[categorical].apply(mutual_info_churn_score)
mutual_info.sort_values(ascending = False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

# Feature importance: Correlation

* Correlation coefficient - https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

In [24]:
df_full_train[numeric].corrwith(df_full_train.churn).abs()

tenure            0.351885
monthlycharges    0.196805
totalcharges      0.196353
dtype: float64

# One Hot Incoding

In [25]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numeric].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numeric].to_dict(orient='records')
X_val = dv.transform(val_dict)

# Logistic regression

In [27]:
xi = np.linspace(-7, 7, 15)
w0 = 4.5
w = np.linspace(0.1, .04, len(xi))

def sigmoid(z):
    return 1/1+np.exp(-z)

def linear_regression(xi):
    z = w0 
    
    for j in range(len(w)):
        z = z+ w0 + w[j]*x[j]
    result = sigmoid(z)
    
    return result

# Training Logistic regression

In [29]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
# find out the bias value:
model.intercept_[0]

-0.10901390050687017

In [33]:
model.coef_[0]

array([ 4.74363276e-01, -1.74973527e-01, -4.07051687e-01, -2.91605758e-02,
       -7.85013623e-02,  6.27871799e-02, -8.88166951e-02, -8.16324229e-02,
       -3.43373810e-02, -7.33245571e-02, -3.35452533e-01,  3.16607290e-01,
       -8.88166951e-02,  3.63409287e-03, -2.58228102e-01,  1.41090236e-01,
        9.47592777e-03,  6.25974472e-02, -8.88166951e-02, -8.14426903e-02,
        2.65861479e-01, -8.88166951e-02, -2.84706722e-01, -2.31681626e-01,
        1.24019688e-01, -1.65604800e-01,  5.79428615e-02, -8.70686838e-02,
       -3.22049378e-02,  7.11331718e-02, -5.95214883e-02,  1.41090236e-01,
       -2.48752175e-01,  2.15585597e-01, -1.20348803e-01, -8.88166951e-02,
        1.01503560e-01, -7.09093929e-02, -8.88166951e-02,  5.20641498e-02,
        2.13838812e-01, -8.88166951e-02, -2.32684055e-01, -7.06342830e-02,
        3.85254037e-04])

In [38]:
y_pred = model.predict_proba(X_val)[:,1]
y_pred

array([0.0089427 , 0.20436801, 0.21223906, ..., 0.13594882, 0.79954066,
       0.83723744])

In [41]:
t = 0.5
churn_decision = (y_pred >= t).astype(int)
churn_decision

array([0, 0, 0, ..., 0, 1, 1])

In [42]:
# create DataFrame

df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = churn_decision
df_pred['actual'] = y_val
df_pred

,probability,prediction,actual
0,0.008943,0,0
1,0.204368,0,0
2,0.212239,0,0
3,0.542899,1,1
4,0.213854,0,0
...,...,...,...
1404,0.312374,0,0
1405,0.039163,0,1
1406,0.135949,0,0
1407,0.799541,1,1


In [44]:
# correct prediction
df_pred['correct'] = (df_pred.prediction == df_pred.actual)
df_pred

,probability,prediction,actual,correct
0,0.008943,0,0,True
1,0.204368,0,0,True
2,0.212239,0,0,True
3,0.542899,1,1,True
4,0.213854,0,0,True
...,...,...,...,...
1404,0.312374,0,0,True
1405,0.039163,0,1,False
1406,0.135949,0,0,True
1407,0.799541,1,1,True


In [46]:
#prediction accuracy
df_pred.correct.mean()

0.8034066713981547

# Model intrepretation
* Look at the coefficients
* Train a smaller model with fewer features

In [55]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'contract=month-to-month': 0.474,
 'contract=one_year': -0.175,
 'contract=two_year': -0.407,
 'dependents=no': -0.029,
 'dependents=yes': -0.079,
 'deviceprotection=no': 0.063,
 'deviceprotection=no_internet_service': -0.089,
 'deviceprotection=yes': -0.082,
 'gender=female': -0.034,
 'gender=male': -0.073,
 'internetservice=dsl': -0.335,
 'internetservice=fiber_optic': 0.317,
 'internetservice=no': -0.089,
 'monthlycharges': 0.004,
 'multiplelines=no': -0.258,
 'multiplelines=no_phone_service': 0.141,
 'multiplelines=yes': 0.009,
 'onlinebackup=no': 0.063,
 'onlinebackup=no_internet_service': -0.089,
 'onlinebackup=yes': -0.081,
 'onlinesecurity=no': 0.266,
 'onlinesecurity=no_internet_service': -0.089,
 'onlinesecurity=yes': -0.285,
 'paperlessbilling=no': -0.232,
 'paperlessbilling=yes': 0.124,
 'partner=no': -0.166,
 'partner=yes': 0.058,
 'paymentmethod=bank_transfer_(automatic)': -0.087,
 'paymentmethod=credit_card_(automatic)': -0.032,
 'paymentmethod=electronic_check': 0.071,

In [56]:
small = ['contract', 'tenure', 'monthlycharges']
df_train[small].iloc[:10].to_dict(orient='records')

[{'contract': 'two_year', 'tenure': 72, 'monthlycharges': 115.5},
 {'contract': 'month-to-month', 'tenure': 10, 'monthlycharges': 95.25},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 75.55},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 80.85},
 {'contract': 'two_year', 'tenure': 18, 'monthlycharges': 20.1},
 {'contract': 'month-to-month', 'tenure': 4, 'monthlycharges': 30.5},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 75.1},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 70.3},
 {'contract': 'two_year', 'tenure': 72, 'monthlycharges': 19.75},
 {'contract': 'month-to-month', 'tenure': 6, 'monthlycharges': 109.9}]

In [57]:
dicts_train_small = df_train[small].to_dict(orient='records')
dicts_val_small = df_val[small].to_dict(orient='records')

In [58]:
dv_small = DictVectorizer(sparse=False)
dv_small.fit(dicts_train_small)


DictVectorizer(sparse=False)

In [59]:
dv_small.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'monthlycharges', 'tenure'], dtype=object)

In [60]:
X_train_small = dv_small.transform(dicts_train_small)

In [61]:
model_small = LogisticRegression(solver='lbfgs')
model_small.fit(X_train_small, y_train)

LogisticRegression()

In [62]:
w0 = model_small.intercept_[0]

w = model_small.coef_[0]
w.round(3)

dict(zip(dv_small.get_feature_names_out(), w.round(3)))

{'contract=month-to-month': 0.97,
 'contract=one_year': -0.025,
 'contract=two_year': -0.949,
 'monthlycharges': 0.027,
 'tenure': -0.036}

In [65]:
# full taining set

dicts_full_train = df_full_train[categorical + numeric].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

y_full_train = df_full_train.churn.values

model = LogisticRegression(solver='lbfgs')
model.fit(X_full_train, y_full_train)

dicts_test = df_test[categorical + numeric].to_dict(orient='records')

In [66]:
X_test = dv.transform(dicts_test)
y_pred = model.predict_proba(X_test)[:, 1]

In [67]:
churn_decision = (y_pred >= 0.5)

In [68]:
(churn_decision == y_test).mean()

0.815471965933286